## Python for REU 2019

_Burt Rosenberg, 29 May 2019_


__compared to as given in the Boot Camp, coalesed Bayes material from the lesson on Classes__


### Supplement: Bayes

This draws heavily from Thinking Bayes by Allen Downey.


> In Baysian statistics, probably is analogous to belief. The higher the probablity the more one believes that the event is the case. It is the probability of the Weather Channel. Tomorrow will occur and it will occur only once &mdash; to say _"50 out of 100 times it was March 22nd, 2019,  it rained"_ makes no sense.

The mantra of Bayesnian statistics is that the subjectivity is captured in the _prior_. But once that subjectivity is accounted for, the rest is objective. Data will update that prior by providing supporting or refuting evidence, and the posterior probability reflects the rational person's belief, given the prior and the evidence.

The formula is, given <code>P(H<sub>i</sub>)</code>, the prior of the probability of <code>H<sub>i</sub></code>, and the occurence <code>D</code>, what is the new belief that <code>H<sub>i</sub></code> is the case? I.e. <code>P(H<sub>i</sub>|D)</code>? This is derived as 

> <code>P(H<sub>i</sub>|D) P(D) = P(D|H<sub>i</sub>) P(H<sub>i</sub>)</code>

the symbol <code>P(D|H<sub>i</sub>)</code> is called the _likelihood_. One avoids calculating the <code>P(D)</code> normalizing factor by normalizing instead normalizing the collection <code>{P(H<sub>i</sub>|D) P(H<sub>i</sub>)}<sub>i</sub></code>, given that the <code>H<sub>i</sub></code> are mutually exclusive and collectively exhaustive.


__The M&amp;M Problem__

Solve the M&M Problem as given in Allen Downey, Think Bayes. See http://www.greenteapress.com/thinkbayes/html/thinkbayes002.html#sec14

In Think Bayes, Prof. Downey gives his own set of classes to calculate solutions in Bayesian statistics. The M&M problem is the problem of deciding the manufacturing year of a bag of M&M's: given two bags of M&M, one from 1994 and one from 1996, but it is unknown which is the '94 and which is the '96, find the likeliest given that a random drawing of one M&M from each bag gave a yellow and a green.

Now, first off, why do we have M&M's from 1994? 

Second, because blue M&M's were introduced in 1995, the color mix inside the bags changed. In extermis, if a blue was drawn from one of the bags, when we know exactly which bag is not from 1994!

This is a case of Bayesian statistics: we have a prior guess &mdash; since the bags came to us randomly it is 50/50 the first is from 1994 and the second from 1996. We get some data: the first bag drew a yellow, the second bag drew and green. Then we update the prior belief to a posterior belief by incorporating this data.

In the language of conditional probability, let H be the case that the first bag is from '94 and the second from 96', and D be the event of a yellow drawn from the first bag and a green from the second. We know P(H) how do we update that to P(H|D)?

Bayes Law is crucial (and from where Bayesian Statistics gets the name). It says:

> P(H|D) = P(D|H) P(H) / P(D)

P(H) is called the prior; P(H|D) the posterior; and P(D|H) the likelihood.



In [1]:
# Fix my broken code


class Likelihood:
    
    def __init__(self,n):
        self.n = n

    def set_prior(self,d):
        if len(d)!=self.n:
            # this shows how to raise in exception
            raise Exception("incorrect dimension")
        # the slice notation d[:] causes a copy, see next lesson
        self.d = d[:]
        
    def set_likelihood(self,lh):
        if len(lh)!=self.n:
            raise Exception("incorrect dimension")
        self.lh = lh[:]
        
    def update_post(self):
        # update the list d to the posterior values
        # do this by producting d[i], the prior, with lh[i], the likelihood
        # then normalizing the result (sum up and divide each by the sum)
        #
        pass
        
    def tell_dist(self):
        return self.d
    
    def tell_likeliest(self):
        # return the index i such that self.d[i] is maxium
        return -1

            
def m_and_m_problem():
    """
    From Allen Downey, Think Bayes
    1994 M&M bag: 30% brown, 20% yellow, 20% red, 10% green, 10% orange, 10% tan
    1996 M&M bag: 24% blue, 20% green, 16% orange, 14% yellow, 13% red, 13% brown
    
    a yellow is drawn from one bag, and a green from the other
    
    what is the likelihood tye yellow came from a 1994 bag?
    """
    
    n = 2  # there are two cases, 0: yellow from 1994, 1: yellow from 1996
    prior = [0.5,0.5]  # we have no idea which case we are in
    likelihood = [
        .2*.2, # if in case 0, independent prob yellow from 1994 and green from 1996
        .1*.14 # if in case 1, independent probl green from 1994 and yellow from 1996
    ]
    
    l_h = Likelihood(n)
    l_h.set_prior(prior)
    l_h.set_likelihood(likelihood)
    l_h.update_post()
    
    # note: using a tuple to return multiple values. tuples are immutable
    return (l_h.tell_dist(),l_h.tell_likeliest())

def test_m_and_m_problem():
    # the answers
    ans = [0.74, 0.25]
    ans_l = 0
    
    # tuple assignment
    (res,likeliest) = m_and_m_problem() 
    
    # shows how to loop over the indices (range(len())), 
    # and how to do list comprehension [ x for la la la ]
    # and how to use max over a list
    err = max([(ans[i]-res[i])**2 for i in range(len(ans))])
    
    if likeliest!=ans_l:
        print("broken!")
    if err<0.01:
        print("correct!")
    else:
        print("broken!")
        
test_m_and_m_problem()


broken!
broken!




__The Train Problem__

We want to estimate the number of trains owned by a certain railroad company, given the observation of trains, and the train number. We know that the company numbers its trains consecutively starting with one. If we observe train 60, for instance, then we know that company owns at least 60 trains. We make several measurements and then provide a guess as to how many trains we haven't seen.

This problem considers two priors. One prior is a uniform distribution over a range from 1 to n. That is, we assume there are some trains, but never more than n, and we have no preference for one number of trains compared to another. This results in some answer. However, things being what they are, it seems silly to think that there is a magic n, and all train companies choose a number uniformly between 1 and n, and that's the number of trains they run. Rather we suspect that the number of trains a company owns follows a power law. And using a power law distribution we find that our estimations are less sensitive to the arbitrary parameter choices we make when choosing a prior.

That's a good thing.

The code demonstrates some features of classes. There is a base class for a Probability Mass Function, and particular PMF's inherit from it. They enhance the class by initilizing a discretized PMF according to either the Uniform or the Power Law distribution.

The Bayes class contains the basic framework for estimation. On creation it is provided a prior, in the form of an initialized PMF object. The exact type of the PMF object will be a subclass, depending on distribution, but all share the class PMF which, and this is the functionality needed for the Bayes class. 

The Bayes class is _abstract_, that is, while it provides a blueprint for methods, the likelihood method is unspecified. That method encodes the particular likelihood function for, say, the Train problem, and therefore the Train class subclasses it and provides the working body of the likelihood method correct for the Train problem.

Which is, by the way, that if the hypothesis says there are n trains, and the train number observed is m, then if m is greater than n, the hypothesis is discarded. It cannot be true. Its probability is set to zero. Otherwise, we give a likelihood of 1/n to the occurence, blindly assuming that of the n trains that we could have been sent our way, any of the n are equally likely to have been sent our way.

Given the distribution of probabilities for each hypothesis, we next consider what our answer will be. The maximum likelihood approach seems inadequate. The solution by maximum likelihood is the maximum number of the observed trains.
So we give two alternatives &mdash; the mean of the distribution and a credible interval, outside of which there is limited probability weight.


<pre>
       +--------+        references       +----------+
       |  PMF   |---------------------+   |   Bayes  |
       +--------+                     |   +----------+
         |    | is-a +-------------+  |        | is-a
         |    +------| Uniform PMF |  |   +-----------+
         | is-a      +-------------+  +---|   Train   |
         |      +---------------+         +-----------+
         +------| Power Law PMF |
                +---------------+
</pre>


__Sample of the solution__

<pre>
num= 1000 obs= [60]
uniform prior 333.4198932637079
credible interval (69, 869)
power law prior 178.5473531797161
credible interval (62, 559)

num= 1000 obs= [30, 60, 90]
uniform prior 164.30558642273346
credible interval (92, 373)
power law prior 133.27523137503107
credible interval (91, 242)

num= 500 obs= [30, 60, 90]
uniform prior 151.84958795903836
credible interval (92, 316)
power law prior 133.27523137503107
credible interval (91, 242)

num= 2000 obs= [30, 60, 90]
uniform prior 171.3381810915096
credible interval (92, 393)
power law prior 133.27523137503107
credible interval (91, 242)
</pre>

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import math
import numpy as np


class Pmf:
    
    def __init__(self,hypoth,prob):
        """
        store a pair of vectors with the hypothesis and the probability fo that hypothesis
        prob will be normalized, so it can be a relative frequency
        hypoth and prob are lists, the initializer will convert to ndarrays
        for the purposes of plotting hypoth are real numbers in increasing value
        """
        assert len(hypoth)==len(prob)
        self.n = 0    # remember the length, that's convenient
        self.hypoth = None # turn hypoth into an ndarray
        self.prob = None # turn prob into an ndarray
        self.normalize()
        
    def normalize(self):
        """
        normalize the prob vector
        """
        pass
    
    def tell_distribution(self):
        """
        return the pair (hypoth,prob), for e.g. plotting the distribution
        """
        return None
    
    def mean(self):
        """
        return the mean of the distribution
        """
        return 0.0
    
    def percentile(self,percent):
        """
        return the hypoth such that the total probability
        of being less that that hypoth is percent
        """
        return None
    
    def credible_interval(self):
        """
        return the pair (h1,h2) such at only 5% of hypothesis are
        less than h1, and 5% of hypothesis are greater than h2
        """
        return (0,0)
        
        
class UniformDist(Pmf):
    """
    a uniform distribution
    """
    def __init__(self,n):
        Pmf.__init__(self,range(1,n+1),[1.0]*n)
        
class PowerLawDist(Pmf):
    """
    a power law distribution
    """
    def __init__(self,n,alpha=1.0):
        self.alpha = alpha
        h = range(1,n+1)
        p = [x**(-alpha) for x in h]
        Pmf.__init__(self,h,p)

class Bayes:
    
    def __init__(self,pmf):
        assert isinstance(pmf,Pmf)
        self.pmf = pmf
    
    def update(self,data):
        """
        update the PMF given observation data
        (1) get the likelihood of data given each hypoth
        (2) multiple each prob by the likelihood
        (3) normalize
        """
        pass
        return self.pmf

    def likelihood(self,data,hypo):
        """
        this makes Bayes abstract. a concrete subclass implements this method
        """
        raise NotImplementedError()
        
class Train(Bayes):
    
    # inherit __init__
    
    def likelihood(self,data,hypo):
        """
        the data is the train number; the hypo is the
        hypothesis there are that many trains
        (1) return 0.0 if data is larger than hypo, 
        because we learn that hypo is false;
        (2) use the uniform prob over hypo number of
        trains otherwise
        """
        pass
        return 0.0

def run_bayes(num,obs):
    
    print("num=",num,"obs=",obs)
    suite = Train(UniformDist(num))
    for o in obs:
        suite.update(o)
    print("uniform prior",suite.pmf.mean())
    print("credible interval",suite.pmf.credible_interval())
    (h,p)= suite.pmf.tell_distribution()
    plt.plot(h,p)

    suite = Train(PowerLawDist(1000))
    for o in obs:
        suite.update(o)
    print("power law prior", suite.pmf.mean())
    print("credible interval",suite.pmf.credible_interval())
    (h,p)= suite.pmf.tell_distribution()

    plt.plot(h,p)
    plt.xlabel("number of trains")
    plt.ylabel("probability")
    plt.legend(["uniform","power law"])

    plt.show()

run_bayes(1000,[60])
run_bayes(1000,[30,60,90])
run_bayes(500,[30,60,90])
run_bayes(2000,[30,60,90])


num= 1000 obs= [60]
uniform prior 0.0
credible interval (0, 0)


TypeError: 'NoneType' object is not iterable